# 使用Bokeh绘制向量场

Bokeh没有提供matplotlib中绘制向量场的函数`quiver()`。我们可以使用`segment()`绘制线段，每个向量箭头由4个点、三条线段绘制。

In [3]:
import numpy as np
from bokeh.io import output_notebook, show
from bokeh.models import ColumnDataSource
from bokeh.plotting import figure
output_notebook()

Loading BokehJS ...

下面的`make_quier()`计算向量场的各个箭头的坐标。其中`x`和`y`是每个箭头的起始坐标，`u`和`v`是箭头的方向和长短。当`scale_uv`为箭头长短的缩放系数，当值为`None`则自动将箭头缩放到箭头起始坐标之间的平均距离。`scale_arrow`为箭头部分占整体的比例，`theta`为箭头与箭杆之间的夹角。

In [36]:
def make_quiver(fig, x, y, u, v, scale_uv=None, scale_arrow=0.4, theta=20):
    x, y, u, v = (np.asanyarray(arr).ravel() for arr in (x, y, u, v))
    if scale_uv is None:
        from scipy.spatial.distance import pdist, squareform
        mean_dist = np.mean(np.sort(squareform(pdist(np.c_[x, y])), axis=1)[:, 1])
        max_len = np.max(np.hypot(u, v))
        scale_uv = mean_dist / max_len
        
    theta = np.deg2rad(theta)
    m1 = np.array([[np.cos(theta), np.sin(-theta)], [np.sin(theta), np.cos(theta)]])
    m2 = np.array([[np.cos(theta), np.sin(theta)], [np.sin(-theta), np.cos(theta)]])
    u, v = u * scale_uv, v * scale_uv
    uv = np.vstack([u, v])
    u1, v1 = m1 @ uv * scale_arrow
    u2, v2 = m2 @ uv * scale_arrow
    data = ColumnDataSource(data=dict(x0=x, y0=y, 
                                      x1=x+u, y1=y+v, 
                                      x2=x+u-u1, y2=y+v-v1, 
                                      x3=x+u-u2, y3=y+v-v2,
                                      u=u, v=v))
    fig.segment(x0="x0", y0="y0", x1="x1", y1="y1", source=data)
    fig.segment(x0="x1", y0="y1", x1="x2", y1="y2", source=data)
    fig.segment(x0="x1", y0="y1", x1="x3", y1="y3", source=data)
    return data

下面是使用`make_quiver()`绘制向量场的一个例子：

In [42]:
a, b, c, d, k, r = 3.2, 0.6, 50, 0.56, 125, 1.6
X, Y = np.mgrid[0:80:10j, 0:80:10j]
U = r * X * (1 - X / k) - a * X * Y / (c + X)
V = b * a * X * Y / (c + X) - d * Y

fig = figure(plot_width=300, plot_height=300)
data = make_quiver(fig, X, Y, U, V)
show(fig)

还可以使用`HoverTool`工具为每个箭头添加提示信息。

In [43]:
from bokeh.models.tools import HoverTool

fig = figure(plot_width=300, plot_height=300)
data = make_quiver(fig, X, Y, U, V)
hover = HoverTool(tooltips=[
    ("(x, y)", "@x0, @y0"),
    ("(u, v)", "@u, @v"),
])
fig.add_tools(hover)
show(fig)